In [1]:
#
# 인터페이스가 간단하면 클래스 대신 함수를 받자
# 

# 파이썬 내장 api의 상당수에는 함수를 넘겨서 동작을 사용자화하는 기능이 있다.
# api는 이런 후크를 이용해서 여러분이 작성한 코드를 실행 중에 호출한다.
# 예를 들어 list 타입의 sort 메소드는 정렬에 필요한 각 인덱스 값을 결정하는 key 인수를 옵셔널하게 받는다.
# 다음 코드에서는 lambda 표현식을 key 후크로 넘겨서 이름 리스트를 길이로 정렬한다.
names = ['Socrates', 'Archimedes', 'Plato', 'Aristotle']
names.sort(key=lambda x: len(x))
print(names)

['Plato', 'Socrates', 'Aristotle', 'Archimedes']


In [10]:
# 파이썬의 후크 중 상당수는 인수와 리턴 값이 잘 정의되어있는 함수다.
# 함수는 클래스보다 설명하기 쉽고 정의하기도 간단해서 후크로 많이 쓰인다.
# 함수가 후크로 동작하는 이유는 파이썬이 일급 함수를 갖췄기 때문이다.

# 예를 들어 defaultdict 클래스의 동작을 사용자화 한다고 가정해보자
# 이 자료구조는 찾을 수 없는 키에 접근할 때마다 호출될 함수를 받는다.
# defaultdict 클래스에 인자로 넘기게될 함수는 딕셔너리에서 찾을 수 없는 대응할 수 없는 값을 반환해야한다.
# 다음은 키를 찾을 수 없을 때마다 로그를 남기고 0을 반환하는 후크를 정의한 코드다
def log_missing():
    print('Key added')
    return 0

# 초기값은 딕셔너리와 원하는 증가 값 리스트로 log_missing 함수를 두번 실행하여 로그를 출력하게 해보자
corrent = {'green':12, 'blue':3}
increments = [
    ('red',5),
    ('blue',17),
    ('orange',9),
]
from collections import defaultdict
result = defaultdict(log_missing, corrent)
print('Before :', dict(result))
for key, amount in increments:
    result[key] += amount
print('After: ', dict(result))

Before : {'green': 12, 'blue': 3}
Key added
Key added
After:  {'green': 12, 'blue': 20, 'red': 5, 'orange': 9}


In [13]:
# log missing 같은 함수를 이용하면 결정 동작과 부작용을 분리하므로 api 설계에 매우 유리하다.
# 기본값 후크를 defaultdict에 넘겨서 찾을 수 없는 키의 총 개수를 세어보자.
def increment_with_report(current, increments):
    added_count = 0
    
    def missing():
        nonlocal added_count
        added_count += 1
        return 0
    result = defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount
    return result, added_count

In [24]:
# defaultdict는 missing 후크가 상태를 유지하는 사실을 모르지만, increment_with_report 함수를 실행하면,
# 튜플의 요소로 기대한 개수인 2를 얻는다.
corrent = {'green':12, 'blue':3}
result, count = increment_with_report(corrent, increments)
assert count == 2

In [21]:
# 상태 보존 후크용으로 클로저를 정의할 때 생기는 문제는 상태가 없는 함수의 예제부다 이해하기 어렵다는 것
# 또 다른 방법은 보존할 상태를 캡슐화하는 작은 클래스를 정의하는 것이다.
class CountMissing:
    def __init__(self):
        self.added = 0
    def missing(self):
        self.added += 1
        return 0
# CountMissing.missing 메소드를 참조해서 defaultdict의 기본값 후크로 넘길 수 있다.

In [27]:
counter = CountMissing()
corrent = {'green':12, 'blue':3}

result = defaultdict(counter.missing, corrent)
for key, amount in increaments:
    result[key] += amount
assert counter.added == 2

In [28]:
# 더 좋은 방법은 __call__함수를 이용하는 것이다.
# __call__은 객체를 함수처럼 호출할 수 있게 해준다.
class BetterCountMissing:
    def __init__(self):
        self.added = 0
    def __call__(self):
        self.added += 1
        return 0
counter = BetterCountMissing()
counter()
assert callable(counter)

In [32]:
counter = BetterCountMissing()
corrent = {'green':12, 'blue':3}
result = defaultdict(counter, corrent)
for key, amount in increaments:
    result[key] += amount
assert counter.added == 2